In [ ]:
%matplotlib inline
import scipy
import matplotlib.pyplot as plt
import librosa
import IPython.display as ipd
import numpy as np

In [ ]:
filename = 'audio/c_strum.wav'
x, sr = librosa.load(filename)

In [ ]:
print(x.shape)
print(sr)

In [ ]:

ipd.Audio(x, rate=sr)

### Introduction to Fourier Transform

Qustion: How can we find out which note has actually been played?
<br>

The pitch of a musical tone is closely related to its fundamental frequency, the frequency of the lowest partial of the sound. Therefore, we need to determine the frequency content, the main periodic oscillations of the signal.

In [ ]:
plt.figure(figsize=(14, 5))
librosa.display.waveshow(x, sr=sr)
plt.grid(True)

In [ ]:
n0 = 17500
n1 = 17800
plt.figure(figsize=(14, 5))
plt.plot(x[n0:n1])
plt.grid(True)
plt.axhline(y=0, color='red', linestyle='--', linewidth=2.5)

The figure shows that the signal behaves in a nearly periodic way within this section. <br>

The main idea of Fourier analysis is to compare the signal with sinusoids of various frequencies ($ω \in R$) (measured in Hz). As a result, we obtain for each considered frequency parameter ($ω \in R$) with a magnitude coefficient ($d_{ω} \in R$)
<br>

The Fourier transform breaks up a signal into its frequency components. For each frequency ($ω \in R$), the Fourier transforms yields a coefficient $d_{ω}$ (and a phase $φ_{ω}$) that tells us to which extent the given signal matches a sinusoidal prototype oscillation of that frequency.
<br>
One important property of the Fourier transform is that the original signal can be reconstructed from the coefficients $d_{ω}$  (along with the coefficients $φ_{ω}$ ).


<br>

The original signal and the Fourier transform contain the same amount of information. This information, however, is represented in different ways. While the signal displays the information across time, the Fourier transform displays the information across frequency.

In [ ]:
import numpy as np
import scipy.fft

# Compute FFT using scipy.fft.fft()
X = scipy.fft.fft(x)
X_mag = np.abs(X)  # Compute magnitude spectrum

# Frequency axis
f = np.linspace(0, sr, len(X_mag))

# Plot result

plt.figure(figsize=(13, 5))
plt.plot(f[:len(f)//2], X_mag[:len(f)//2])  # Show only positive frequencies
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.title("FFT of the Signal")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(f[:5000], X_mag[:5000])
plt.xlabel('Frequency (Hz)')
plt.grid(True)

### Discrete Fourier Transform (DFT)

Before jumping into FFT, we're going to introduce DFT conceptually and mathematically.
<br>

Computing the Fourier transform of signals involves the evaluation of integrals or infinite sums, which is, in general, computationally infeasible. In practice, one typically approximates the Fourier transform by finite sums. Furthermore, the Fourier transform is evaluated only for a finite number of frequencies.
<br>

In this part of the class, we show how the finite sums and the Fourier co-efficients must be chosen to obtain a linear transform known as the discrete Fourier transform (DFT)
<br>

The important point is that the DFT can be computed efficiently by means of an algorithm, the famous fast Fourier transform (FFT).


### Understanding the Discrete Fourier Transform (DFT)

#### What is the DFT?
The Discrete Fourier Transform (DFT) converts a discrete time-domain signal into its frequency-domain representation. This transformation allows us to analyze which frequencies make up a signal.

#### Mathematical Definition
Given a discrete signal $x[n]$ of length $N$, the DFT is defined as:

\begin{align}
X[k] = \sum_{n=0}^{N-1} x[n] e^{-j 2\pi k n / N}
\end{align}

where:
*   $X[k]$, DFT coefficient, represents the frequency content at index $k$.
*   $e^{-j 2\pi k n / N}$ is a complex exponential, which acts like a "frequency probe" to extract how much of a given frequency is present in
*   The output is a vector $X$ of length $N$, which contains complex numbers representing both magnitude (strength) and phase (timing shift) of each frequency.


<br>

To better understand the DFT, consider the basis function:
\begin{align}
u_{k}[n] = e^{j 2\pi k n / N} = \cos(2\pi k n / N) + j \sin(2\pi k n / N)
\end{align}



*   This is a sampled complex sinusoid at frequency $k$/$N$.
*   The DFT essentially projects the signal $x[𝑛]$ onto these complex sinusoids to determine how much of each frequency component is present.
*   The resulting value $X[k]$ is the inner product (or similarity measure) between $x[n]$ and $u_{k}[n]$:

\begin{align}
X[k] = \langle x | u_{k} \rangle
\end{align}

where:

*   $Re(X[k])$ (real part) measures similarity with the cosine component.
*   $Im(X[k])$ (imaginary part) measures similarity with the sine component.
*   $|X[k]|$ (magnitude) tells us how strongly the frequency $k$ is present.









In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline


N = 64
n = np.arange(N)
k = 3
x = np.cos(2 * np.pi * (k * n / N) + (1.2*np.random.rand(N) - 0.0))

plt.figure(figsize=(10, 5))

plt.subplot(2, 1, 1)
plt.plot(n, x, 'k', marker='.', markersize='10', linewidth=2.0, label='$x$')
plt.xlabel('Time (samples)')
k = 3
u_k_real = np.cos(2 * np.pi * k * n / N)
u_k_imag = -np.sin(2 * np.pi * k * n / N)
u_k = u_k_real + u_k_imag*1j
sim_complex = np.vdot(u_k, x)
sim_abs = np.abs(sim_complex)
plt.title(r'Signal $x$ and some $u_k$ (k=3) having high similarity: Re($X(k)$) = %0.2f, Im($X(k)$) = %0.2f,  $|X(k)|$=%0.2f'%(sim_complex.real,sim_complex.imag,sim_abs))
plt.plot(n, u_k_real, 'r', marker='.', markersize='5',
         linewidth=1.0, linestyle=':', label='$\mathrm{Re}(\overline{\mathbf{u}}_k)$');
plt.plot(n, u_k_imag, 'b', marker='.', markersize='5',
         linewidth=1.0, linestyle=':', label='$\mathrm{Im}(\overline{\mathbf{u}}_k)$');
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(n, x, 'k', marker='.', markersize='10', linewidth=2.0, label='$x$')
plt.xlabel('Time (samples)')
k = 5
u_k_real = np.cos(2 * np.pi * k * n / N)
u_k_imag = -np.sin(2 * np.pi * k * n / N)
u_k = u_k_real + u_k_imag*1j
sim_complex = np.vdot(u_k, x)
sim_abs = np.abs(sim_complex)
plt.title(r'Signal $x$ and some $u_k$ (k=5) having low similarity: Re($X(k)$) = %0.2f, Im($X(k)$) = %0.2f,  $|X(k)|$=%0.2f'%(sim_complex.real,sim_complex.imag,sim_abs))
plt.plot(n, u_k_real, 'r', marker='.', markersize='5',
         linewidth=1.0, linestyle=':', label='$\mathrm{Re}(\overline{\mathbf{u}}_k)$');
plt.plot(n, u_k_imag, 'b', marker='.', markersize='5',
         linewidth=1.0, linestyle=':', label='$\mathrm{Im}(\overline{\mathbf{u}}_k)$');
plt.legend()

plt.tight_layout()

The plot above helps us understand how the Discrete Fourier Transform (DFT) identifies the presence of specific frequencies in a signal by comparing it with basis functions.


*   The black line represents the signal 𝑥, which consists of a noisy cosine wave with a frequency component at 𝑘=3.

*   The red dashed line represents the real part of the basis function $u_{k}$, which is a cosine wave at frequency $k$.

*   The blue dashed line represents the imaginary part of $u_{k}$, which is a sine wave at frequency $k$.

<br>

So what does the two plot show?

<br>

The first plot (k = 3, high similarity):


*   The basis function at $k=3$ closely matches the signal $x$, meaning the DFT detects a strong frequency component at this index.
*   The real and imaginary parts of $X(k)$ are large, and the magnitude $|X(k)|$ is high, indicating strong frequency content at $k=3$

The first plot (k = 5, low similarity):


*   The basis function at $k=5$ does not aling well with $x$, meaning the signal does not have much energy at this frequency.

*   The real and imaginary parts of $X(k)$ are smaller, and the magnitude $|X(k)|$ is low, indicating little presence of frequency $k=5$ in the signal.


### Key Takeaways

*  The DFT works by measuring how well a signal aligns with different frequency basis functions.
*  If a signal contains a strong frequency component, the corresponding DFT coefficient $X(k)$ will be large.
*  If a frequency is not present in the signal, the DFT coefficient will be small.





### Summary

The Discrete Fourier Transform (DFT) is like a musical detective that breaks down a signal into its individual frequency components, telling us how much of each frequency is present and how they are phased together.
<br>

Here's an intuitive way to think about it:

*   Time vs. Frequency: A signal in the time domain is like hearing a song played all at once, while the frequency domain is like identifying each individual note in the song.
*   DFT as Pattern Matching: The DFT compares the input signal to a set of perfectly tuned sine and cosine waves (basis functions) at different frequencies. If the signal has a strong match with a certain frequency, the corresponding DFT output is large.
*   Real and Imaginary Parts: The DFT uses both cosine (real part) and sine (imaginary part) to capture phase shifts in the signal. This tells us not only what frequencies exist but when they occur relative to each other.
*   Magnitude and Phase:
    *   The magnitude tells us how strong a frequency component is.
    *   The phase tells us where that frequency occurs in time.
*   Understanding Inner Products: The DFT is essentially measuring how much of each frequency component is present in the signal by computing an inner product (dot product) with sinusoids. If they align well, we get a strong response.
*   Periodic Signals & Harmonics: If the signal is periodic (like a musical note), the DFT will show strong peaks at specific harmonics, helping us analyze the fundamental frequency and overtones.


### Fast Fourier Transform (FFT)

#### Why do we need FFT?
The DFT is computationally expensive. Why?
<br>

The DFT is Like Matching a Song to Many Reference Tunes

*   magine you have a song (your time-domain signal) and a playlist of $N$ different reference melodies (your basis functions).
*   To figure out if the song contains a certain melody (frequency component), you must compare the song to every single reference tune.
*   This means for each reference melody (each frequency $k$), you have to:
Listen to the entire song (sum over all $N$ samples). Compare each note to see how much it matches.
*   Since we need to repeat this for every melody (frequency component $k$), this means for a 10,000-sample audio signal, we'd have to do 100,000,000 calculations! 😱 That's way too slow, especially for real-time music processing.


#### How FFT Solves the Problem

Instead of brute-force checking each reference melody against the whole song, what if we could break the problem into smaller, easier chunks? This is exactly what the Fast Fourier Transform (FFT) does.


In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

# Function to compute DFT manually (Brute-force method)
def dft(x):
    """
    Compute the Discrete Fourier Transform (DFT) of a signal using a brute-force method.
    """
    N = len(x)
    X = np.zeros(N, dtype=complex)
    for k in range(N):
        for n in range(N):
            X[k] += x[n] * np.exp(-2j * np.pi * k * n / N)
    return X

# Generate a test signal (sum of two sine waves)
N = 1024  # Number of samples
t = np.linspace(0, 1, N, endpoint=False)  # Time axis
signal = np.sin(2 * np.pi * 50 * t) + 0.5 * np.sin(2 * np.pi * 120 * t)  # 50Hz and 120Hz components

# Compute DFT and measure execution time
start_time = time.time()
X_dft = dft(signal)
dft_time = time.time() - start_time

# Compute FFT using NumPy and measure execution time
start_time = time.time()
X_fft = np.fft.fft(signal)
fft_time = time.time() - start_time

# Plot the magnitude of DFT vs. FFT
freqs = np.fft.fftfreq(N, d=1/N)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(freqs[:N//2], np.abs(X_dft[:N//2]), label="DFT (Brute-force)")
plt.title(f"DFT Computation (Time: {dft_time:.4f}s)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(freqs[:N//2], np.abs(X_fft[:N//2]), label="FFT (Fast Algorithm)", color='r')
plt.title(f"FFT Computation (Time: {fft_time:.6f}s)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.legend()

plt.tight_layout()
plt.show()

# Print efficiency comparison
print(f"DFT Computation Time: {dft_time:.4f} seconds")
print(f"FFT Computation Time: {fft_time:.6f} seconds")
print(f"Speed Improvement: {dft_time / fft_time:.0f}x Faster!")


### Missing Time Localization
The Fourier transform yields frequency information that is averaged over the entire time domain. However, the information on when these frequencies occur is hidden in the transform. This phenomenon is illustrated by the following example.

In [ ]:
Fs = 128
duration = 10
omega1 = 1
omega2 = 5
N = int(duration * Fs)
t = np.arange(N) / Fs
t1 = t[:N//2]
t2 = t[N//2:]

x1 = 1.0 * np.sin(2 * np.pi * omega1 * t1)
x2 = 0.7 * np.sin(2 * np.pi * omega2 * t2)
x = np.concatenate((x1, x2))

plt.figure(figsize=(8, 2))
plt.subplot(1, 2, 1)
plt.plot(t, x, c='k')
plt.xlim([min(t), max(t)])
plt.xlabel('Time (seconds)')

plt.subplot(1, 2, 2)
X = np.abs(np.fft.fft(x)) / Fs
freq = np.fft.fftfreq(N, d=1/Fs)
X = X[:N//2]
freq = freq[:N//2]
plt.plot(freq, X, c='k')
plt.xlim([0, 7])
plt.ylim([0, 3])
plt.xlabel('Frequency (Hz)')
plt.tight_layout()

### Basic Idea
To recover the hidden time information, Dennis Gabor introduced in the year 1946 the short-time Fourier transform (STFT). Instead of considering the entire signal, the main idea of the STFT is to consider only a small section of the signal. To this end, one fixes a so-called window function, which is a function that is nonzero for only a short period of time (defining the considered section). The original signal is then multiplied with the window function to yield a windowed signal. To obtain frequency information at different time instances, one shifts the window function across time and computes a Fourier transform for each of the resulting windowed signals. This idea is illustrated by the next example.

In [ ]:
from ipywidgets import interact, fixed, FloatSlider
def windowed_ft(t, x, Fs, w_pos_sec, w_len):

    N = len(x)
    w_pos = int(Fs * w_pos_sec)
    w_padded = np.zeros(N)
    w_padded[w_pos:w_pos + w_len] = 1
    x = x * w_padded
    plt.figure(figsize=(8, 2))

    plt.subplot(1, 2, 1)
    plt.plot(t, x, c='k')
    plt.plot(t, w_padded, c='r')
    plt.xlim([min(t), max(t)])
    plt.ylim([-1.1, 1.1])
    plt.xlabel('Time (seconds)')

    plt.subplot(1, 2, 2)
    X = np.abs(np.fft.fft(x)) / Fs
    freq = np.fft.fftfreq(N, d=1/Fs)
    X = X[:N//2]
    freq = freq[:N//2]
    plt.plot(freq, X, c='k')
    plt.xlim([0, 7])
    plt.ylim([0, 3])
    plt.xlabel('Frequency (Hz)')
    plt.tight_layout()
    plt.show()

w_len = 4 * Fs
windowed_ft(t, x, Fs, w_pos_sec=1, w_len=w_len)
windowed_ft(t, x, Fs, w_pos_sec=3, w_len=w_len)
windowed_ft(t, x, Fs, w_pos_sec=5, w_len=w_len)

print('Interactive interface for experimenting with different window shifts:')
interact(windowed_ft,
         w_pos_sec=FloatSlider(min=0, max=duration-(w_len/Fs), step=0.1,
                continuous_update=False, value=1.7, description='Position'),
                t=fixed(t), x=fixed(x), Fs=fixed(Fs), w_len=fixed(w_len));

It is important to note that the STFT reflects not only the properties of the original signal but also those of the window function. First of all, the STFT depends on the length of the window, which determines the size of the section. Then, the STFT is influenced by the shape of the window. For example, the sharp edges of the rectangular window typically introduce "ripple" artifacts. We discuss such issues in more detail later.

### Formal Definition of the Discrete STFT

#### Intuitive Analogy: STFT as a "Sliding Magnifying Glass"

*   Imagine you have a song (your signal) written on a long scroll.
*   You can't read the entire song at once, so you use a magnifying glass to focus on small sections at a time.
*   As you slide the magnifying glass across the scroll, you analyze each section separately before moving to the next.
*   This is exactly what STFT does: it applies FFT on small overlapping chunks of the signal instead of analyzing the whole thing at once.


#### Breaking down the STFT Formula
\begin{align}
X(m, k) = \sum_{n=0}^{N-1} x(n + mH) w(n) e^{-2\pi i \frac{kn}{N}}
\end{align}

Let's break this down steo by step:


*   $x(n)$: The discrete-time signal (the raw audio samples).
*   $w(n)$: A window function that extracts a small portion of the signal.
*   $N$: Window length (number of samples per segment).
*   $H$: Hop size, the number of samples the window shifts per step.
*   $m$: Frame index (which time window we're analyzing).
*   $k$: Frequency bin index (which frequency we're analyzing).
*   $e^{-2\pi i \frac{kn}{N}}$: The complex exponential term from DFT (extracts frequency components).

What's Happening Conceptually?
*   The window function $w(n)$ extracts a small segment of the signal $x(n)$.
*   This window moves forward by $H$ samples per step, capturing different time slices.
*   For each time slice, FFT is applied to compute its frequency content.
*   The result is a matrix $X(m,k):$
  *   $m$: Time (which window we're analyzing).
  *   $k$: Frequency (which frequency component is present at that time).







### Librosa.stft


```
librosa.stft(y, *, n_fft=2048, hop_length=None, win_length=None, window='hann',
             center=True, dtype=None, pad_mode='constant', out=None)
```




*   `y`: input signal
*   `n_fft`: Number of FFT points
*   `hop_length`: The number of samples between successive frames.
*   `win_length`: The actual size of the window function (in samples). If not set, it defaults to n_fft.
*   `window`: The windowing function to reduce spectral leakage.
*   `center`: Pads the signal so that the window is centered at each frame.

<br>

`n_fft`: stands for Number of FFT points, which means:
*   It's the length of the windowed signal segment that is transformed into the frequency domain.
*   Determines how many discrete points are used to compute the FFT for each segment.
* If `n_fft` is 1024, it means each FFT is computed on 1024 samples of the signal at a time.
* The result will have `n_fft/2 + 1` frequency bins because the output is symmetrical (for real-valued signals).

<br>

Relationship Between n_fft and window_length:

*   In STFT, the signal is divided into windows, and each window is transformed into the frequency domain using the FFT.
*   n_fft is the number of points used in the FFT calculation, but it can be different from the window_length, which is the actual size of the segment of the signal you're analyzing.










In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

# Load an example audio file
y, sr = librosa.load(librosa.example('trumpet'), sr=22050)

# Parameters to test
params = [
    (512, 512),  # n_fft = window_length
    (1024, 512), # n_fft > window_length (zero-padding)
    # (256, 512)   # window_length > n_fft (uncommon)
]

plt.figure(figsize=(15, 8))

for i, (n_fft, win_length) in enumerate(params):
    stft_result = librosa.stft(y, n_fft=n_fft, win_length=win_length, hop_length=256)
    spectrogram = librosa.amplitude_to_db(np.abs(stft_result), ref=np.max)

    plt.subplot(1, 3, i+1)
    librosa.display.specshow(spectrogram, sr=sr, hop_length=256, x_axis='time', y_axis='log')
    plt.colorbar()
    plt.title(f'n_fft={n_fft}, win_length={win_length}')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')

plt.tight_layout()
plt.show()


* If n_fft = window_length:
 * FFT is computed directly on the windowed signal.
* If n_fft > window_length:
 * The windowed signal is zero-padded to match the n_fft size, which can improve frequency resolution.
* If n_fft < window_length:
 * This is uncommon because you lose data, but technically possible.

In [ ]:
# Load an audio file
filename = librosa.example('trumpet')  # You can replace this with any local audio file
y, sr = librosa.load(filename, sr=22050)

# Compute the Short-Time Fourier Transform
n_fft = 1024
hop_length = 256
stft_result = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)

# Get the frequency values for each FFT bin
freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)

# Select specific frames to plot (e.g., every 20th frame)
frames_to_plot = np.arange(0, stft_result.shape[1], 20)

plt.figure(figsize=(10, 6))

for frame in frames_to_plot:
    magnitude_spectrum = np.abs(stft_result[:, frame])  # Magnitude for the current frame
    plt.plot(freqs, magnitude_spectrum, label=f'Frame {frame}')

plt.title("Frequency vs. Magnitude for Selected STFT Frames")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.legend()
plt.show()



In [ ]:
# Load an audio file
filename = librosa.example('trumpet')  # Replace with your audio file
y, sr = librosa.load(filename, sr=22050)

# Compute the Short-Time Fourier Transform
n_fft = 1024
hop_length = 256
stft_result = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)

# Get the frequency and time values
freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
times = librosa.times_like(stft_result, sr=sr, hop_length=hop_length)

# Create the figure and 3D axis
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot each frame as a separate line in 3D space
for i in range(len(times)):
    ax.plot(freqs, [times[i]] * len(freqs), np.abs(stft_result[:, i]))

# Set axis labels
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Time (s)')
ax.set_zlabel('Magnitude')
ax.set_title('3D Plot of Frequency vs. Magnitude over Time (STFT)')

plt.show()